<a href="https://colab.research.google.com/github/RaczeQ/networkx-library-workshop/blob/master/notebooks/NetworkX_Live.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load files

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import collections
from operator import itemgetter

import networkx as nx

## NetworkX Methods

### Loading network

Adjacency list - `nx.read_adjlist(path, comments='#')`

Edge list - `nx.read_edgelist(path, comments='#')`

### Generating network

Complete - `nx.complete_graph(nodes)`

Balanced tree - `nx.balanced_tree(branching_factor, height)`

Erdős-Rényi  - `nx.erdos_renyi_graph(nodes, probability, seed=None, directed=False)`

Barabási–Albert - `barabasi_albert_graph(nodes, edges_from_node, seed=None)`

### Measures

Betweenness - `nx.betweenness_centrality(graph)`

Closeness - `nx.closeness_centrality(graph)`

Clustering coefficient - `nx.clustering(graph)`

Degree - `nx.degree(graph)`

Density - `nx.betweenness_centrality(graph)`

Diameter - `nx.diameter(graph)` 
(Caution: unconnected graph raises exception)

Number of connected components - `nx.number_connected_components(graph)`

Pagerank - `nx.pagerank(graph)`

Shortest path length - `nx.average_shortest_path_length(graph)`

### Manipulating network

Add node - `graph.add_node(node)`

Add nodes - `graph.add_nodes_from([node1, node2, ...])` / `graph.add_nodes_from(range(1,100))`

Add edge - `graph.add_edge(node1, node2)`

Attributes:

```
>>> graph.add_node(node, attribute_name=value, time=5, weather="sunny")
>>> graph.nodes[node][attibute_name]
value
>>> del graph.nodes[node][attibute_name]
>>> graph.nodes[node]
{ "time": 5, "weather": "sunny"}
```



In [ ]:
barabasi = nx.barabasi_albert_graph(100, 5)
print(nx.info(barabasi))

In [ ]:
nx.draw(barabasi)

In [ ]:
degree = nx.degree(barabasi)
print(degree)

In [ ]:
average_degree = np.mean([d for n,d in degree])
print(average_degree)

In [ ]:
(largest_hub, d) = sorted(degree, key=itemgetter(1))[-1]
# Create ego graph of main hub
hub_ego = nx.ego_graph(barabasi, largest_hub)
# Draw graph
pos = nx.spring_layout(hub_ego)
nx.draw(hub_ego, pos, node_color='b', node_size=50, with_labels=False)
# Draw ego as large and red
nx.draw_networkx_nodes(hub_ego, pos, nodelist=[largest_hub], node_size=300, node_color='r')
plt.show()

In [ ]:
degree_sequence = sorted([d for n, d in degree], reverse=True)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

plt.bar(deg, cnt)

In [ ]:
jazz_graph = nx.read_adjlist('../networks/jazz', comments='%')

In [ ]:
print(nx.info(jazz_graph))

In [ ]:
nx.draw(jazz_graph)

In [ ]:
plt.figure(figsize=(50, 50))
nx.draw_circular(jazz_graph)

In [ ]:
plt.figure(figsize=(30, 30))
nx.draw_kamada_kawai(jazz_graph)

In [ ]:
plt.figure(figsize=(30, 30))
nx.draw_random(jazz_graph)

In [ ]:
degree = nx.degree(jazz_graph)
print(degree)

In [ ]:
average_degree = np.mean([d for n,d in degree])
print(average_degree)

In [ ]:
degree_sequence = sorted([d for n, d in degree], reverse=True)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

plt.figure(figsize=(8, 8))
plt.bar(deg, cnt)
plt.draw()

In [ ]:
betweenness = nx.betweenness_centrality(jazz_graph)
betweenness

In [ ]:
average_betweenness = np.mean([b for d,b in betweenness.items()])
print(average_betweenness)

In [ ]:
pagerank = nx.pagerank(jazz_graph)
print(pagerank)
print('Max pagerank:', sorted([p for n, p in pagerank.items()], reverse=True)[0])

## Visualization

In [ ]:
karate_club = nx.karate_club_graph()
print(nx.info(karate_club))

### Matplotlib

In [ ]:
nx.draw(karate_club)

In [ ]:
nx.draw(karate_club, pos=nx.circular_layout(karate_club))

In [ ]:
pos = nx.circular_layout(karate_club, karate_club.nodes())
nx.draw(karate_club, pos, with_labels=True)

In [ ]:
pos = nx.circular_layout(karate_club, list(map(lambda node: 2*node if node % 3 == 0 else node,[node for node in karate_club.nodes()])))
nx.draw(karate_club, pos, with_labels=True)

In [ ]:
pos = nx.spring_layout(karate_club, iterations=200)
nx.draw(karate_club, node_color=range(34), node_size=200, cmap=plt.cm.Greens, pos=pos)

In [ ]:
first_kc_group = list(karate_club.nodes())[:15]
color_list = ["red" for _ in range(15)] + ["blue" for _ in range(len(karate_club) - 15)]

pos = nx.bipartite_layout(karate_club, first_kc_group)
plt.figure(figsize=(30, 30))
nx.draw(karate_club, node_color=color_list, node_size=200, pos=pos)

In [ ]:
nx.draw_random(karate_club, node_color="blue", node_size=400, cmap=plt.cm.Greens)

### D3.js

In [ ]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json

json_network = nx.node_link_data(karate_club)
print(json_network)

In [ ]:
css_text = '''
.links line {
    stroke: #343434;
    stroke-opacity: 0.6;
}

.nodes circle {
    stroke: #ff7058;
    stroke-width: 2px;
    fill: rgb(132, 219, 255);
}

'''

In [ ]:
js_text_template = Template('''

var data = '$python_data' ;
var id = '$id';

console.log(data);

chart = {
  const links = data.links.map(d => Object.create(d));
  const nodes = data.nodes.map(d => Object.create(d));

  const simulation = d3.forceSimulation(nodes)
      .force("link", d3.forceLink(links).id(d => d.id))
      .force("charge", d3.forceManyBody())
      .force("x", d3.forceX())
      .force("y", d3.forceY());

  const svg = d3.select(DOM.svg(width, height))
      .attr("viewBox", [-width / 2, -height / 2, width, height]);

  const link = svg.append("g")
      .attr("stroke", "#999")
      .attr("stroke-opacity", 0.6)
    .selectAll("line")
    .data(links)
    .join("line")
      .attr("stroke-width", d => Math.sqrt(d.value));

  const node = svg.append("g")
      .attr("stroke", "#fff")
      .attr("stroke-width", 1.5)
    .selectAll("circle")
    .data(nodes)
    .join("circle")
      .attr("r", 5)
      .attr("fill", color)
      .call(drag(simulation));

  node.append("title")
      .text(d => d.id);

  simulation.on("tick", () => {
    link
        .attr("x1", d => d.source.x)
        .attr("y1", d => d.source.y)
        .attr("x2", d => d.target.x)
        .attr("y2", d => d.target.y);

    node
        .attr("cx", d => d.x)
        .attr("cy", d => d.y);
  });

  invalidation.then(() => simulation.stop());

  return svg.node();
}

''')

In [ ]:
js_template2 = Template('''
var drawKarateClub = function () {
    var data = '$python_data' ;
    var id = '$id';
    var svg = d3.select("svg"),
        width = +svg.attr("width"),
        height = +svg.attr("height");

    var color = d3.scaleOrdinal(d3.schemeCategory10);

    var simulation = d3.forceSimulation()
        .force("link", d3.forceLink().id(function (d) {
            return d.id;
        }))
        .force("charge", d3.forceManyBody())
        .force("center", d3.forceCenter(width / 2, height / 2));

    var graph = JSON.parse(data);

    var link = svg.append("g")
        .attr("class", "links")
        .selectAll("line")
        .data(graph.links)
        .enter().append("line")
        .attr("stroke-width", function (d) {
            return Math.sqrt(d.value);
        });

    var node = svg.append("g")
        .attr("class", "nodes")
        .selectAll("circle")
        .data(graph.nodes)
        .enter().append("circle")
        .attr("r", 7)
        .call(d3.drag()
            .on("start", dragstarted)
            .on("drag", dragged)
            .on("end", dragended));
    node.append("title")
        .text(function (d) {
            return d.id;
        });

    node.attr('id', function (d) {
        return 'id' + d.id;
    });

    simulation
        .nodes(graph.nodes)
        .on("tick", ticked);

    simulation.force("link")
        .links(graph.links);

    function ticked() {
        link
        .attr("x1", function (d) {
                return d.source.x;
            })
            .attr("y1", function (d) {
                return d.source.y;
            })
            .attr("x2", function (d) {
                return d.target.x;
            })
            .attr("y2", function (d) {
                return d.target.y;
            });

        node
            .attr("cx", function (d) {
                return d.x;
            })
            .attr("cy", function (d) {
                return d.y;
            });
    }

    function dragstarted(d) {
        if (!d3.event.active) simulation.alphaTarget(0.3).restart();
        d.fx = d.x;
        d.fy = d.y;
    }

    function dragged(d) {
        d.fx = d3.event.x;
        d.fy = d3.event.y;
    }

    function dragended(d) {
        if (!d3.event.active) simulation.alphaTarget(0);
        d.fx = null;
        d.fy = null;
    }
};
drawKarateClub();
''')

In [ ]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div">
    <h5>$title</h5>
    <svg width="800" height="300"></svg>                
</div>
<script>
    $js_text
</script>
''')

In [ ]:
%load_ext py_d3

In [ ]:


#with open('d3.min.js', 'r') as myfile:
#  js_data = myfile.read()

#d3 = 'console.log("ASDA");' + js_data

json_graph = json.dumps(json_network)
json_graph = json_graph.replace("'", r"\'")
js_text = js_template2.substitute({'python_data': json_graph, 'id': 'graph-div' })
html = HTML(html_template.substitute({'title':'Zachary\'s Karate Club','css_text': css_text, 'js_text': js_text}))
display(html)

## Generatory grafów

### Klasyczne (proste) generatory

In [ ]:
empty_graph = nx.empty_graph(20, create_using=nx.Graph)
nx.draw(empty_graph)

In [ ]:
null_graph = nx.null_graph(create_using=nx.DiGraph)
print(nx.info(null_graph))

In [ ]:
complete_graph = nx.complete_graph(15, create_using=nx.DiGraph)
nx.draw(complete_graph)

In [ ]:
cycle_graph= nx.cycle_graph(20)
nx.draw(cycle_graph)

In [ ]:
pos = nx.circular_layout(cycle_graph)
nx.draw(cycle_graph, pos)

In [ ]:
star_graph = nx.star_graph(10)
degrees = nx.degree(star_graph)
pos = nx.spring_layout(star_graph)
nx.draw(star_graph, node_color='green', node_size=[degree*100 for (node, degree) in degrees])

In [ ]:
star_link_data = nx.node_link_data(star_graph)
json_graph = json.dumps(star_link_data)
json_graph = json_graph.replace("'", r"\'")
js_text = js_template2.substitute({'python_data': json_graph, 'id': 'graph-div' })
html = HTML(html_template.substitute({'title': 'Star graph', 'css_text': css_text, 'js_text': js_text}))
display(html)

In [ ]:
grid_graph = nx.grid_graph(dim=[5, 10])
pos = nx.spectral_layout(grid_graph)
nx.draw(grid_graph, pos, node_color=range(50), node_size=800, cmap=plt.cm.Purples)

In [ ]:
cubical_graph = nx.cubical_graph()
nx.draw(cubical_graph, node_color="orange")

In [ ]:
diamond_graph = nx.diamond_graph()
nx.draw(diamond_graph, node_size=1200)

### Random graphs

#### Newman–Watts–Strogatz small-world graph

In [ ]:
nws_graph = nx.newman_watts_strogatz_graph(n=10, k=2, p=0.5, seed=2)
nx.draw(nws_graph, node_color="pink")

#### Barabási–Albert preferential attachment model graph

In [ ]:
barabasi_graph = nx.barabasi_albert_graph(n=20, m=7, seed=2)
pos = nx.spring_layout(barabasi_graph)

plt.figure(figsize=(30, 30))

nodes = nx.draw_networkx_nodes(barabasi_graph, pos, node_color="yellow", node_size=1000)
nodes.set_edgecolor('r')

nx.draw_networkx_edges(barabasi_graph, pos, edge_color="purple")

degree=nx.degree(barabasi_graph)
nx.draw_networkx_labels(barabasi_graph, labels=dict(degree), pos=pos)
plt.show()